In [6]:
from poketypes.showdown import BattleMessage, BMType
import pandas as pd
from tqdm.auto import tqdm
import re

In [7]:
bracket_terms = [
    'anim',
    'block',
    'broken',
    'consumed',
    'damage',
    'eat',
    'fail',
    'fatigue',
    'from',
    'fromitem',
    'identify',
    'miss',
    'move',
    'msg',
    'notarget',
    'of',
    'partiallytrapped',
    'silent',
    'source',
    'spread',
    'still',
    'true',
    'upkeep',
    'weak',
    'weaken',
    'wisher'
]

filter_subset = [
    "MessageCategory", "|Count", ":Count", "last|"
] + [f"[{bt}]" for bt in bracket_terms]

In [8]:
def create_messagetypes(gen: int):
    formats = [
        f"gen{gen}randombattle",
        f"gen{gen}randomdoubles"
    ]
    df_list = []
    
    for format_name in formats:
        print("---------------------------")
        print(f"Working on {format_name} now!")
        
        print("Reading lines from log...")
        with open(f"../logs/{format_name}_challenge_20000.log", "r", encoding="utf8") as f:
            lines = [l.strip() for l in f.readlines() if len(l.strip()) > 1]
        
        print(f"Found {len(lines)} lines")
        
        df = pd.DataFrame(columns=["RawMessage"], data=lines)

        print("Dropping Full duplicates...")
        df = df.drop_duplicates().reset_index(drop=True)

        print("Running filter categories...")
        with tqdm(total=4+len(bracket_terms)) as pbar:
            df["MessageCategory"] = df["RawMessage"].str.split("|").str[1]
            pbar.update(1)
            df["|Count"] = df["RawMessage"].str.count("\\|")
            pbar.update(1)
            df[":Count"] = df["RawMessage"].str.count(":")
            pbar.update(1)
            df["last|"] = df["RawMessage"].str[-1] == "|"
            pbar.update(1)
            for bracket_term in bracket_terms:
                df[f"[{bracket_term}]"] = df["RawMessage"].str.contains(f"\\[{bracket_term}\\]")
                pbar.update(1)
        
        print("Dropping Filter duplicates...")
        df = df.drop_duplicates(subset=filter_subset).reset_index(drop=True)
        df_list.append(df)
    
    print(f"Merged all formats for gen {gen}!")
    df = pd.concat(df_list, ignore_index=True)
    df = df.drop_duplicates(subset=filter_subset).reset_index(drop=True)
    df = df[["RawMessage", "MessageCategory"]]
    
    df.to_csv(f"../logs/gen{gen}_messagetypes.csv", sep="\t", index=False)

    return df

df = create_messagetypes(1)
df

---------------------------
Working on gen1randombattle now!
Reading lines from log...
Found 9806719 lines
Dropping Full duplicates...
Running filter categories...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:57<00:00,  1.92s/it]


Dropping Filter duplicates...
---------------------------
Working on gen1randomdoubles now!
Reading lines from log...
Found 6643405 lines
Dropping Full duplicates...
Running filter categories...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:38<00:00,  1.29s/it]


Dropping Filter duplicates...
Merged all formats for gen 1!


,RawMessage,MessageCategory
0,|init|battle,init
1,|title|colress-gpt vs. colress-gpt-test1,title
2,|j|☆colress-gpt,j
3,|request|,request
4,|t:|1695993683,t:
...,...,...
75,"|request|{""active"":[{""moves"":[{""move"":""Sky Att...",request
76,"|request|{""active"":[{""moves"":[{""move"":""Transfo...",request
77,"|request|{""active"":[{""moves"":[{""move"":""Rest"",""...",request
78,"|request|{""active"":[{""moves"":[{""move"":""Recharg...",request


In [5]:
df = df.sort_values(by=["MessageCategory", "Generation"])
# df = df.drop_duplicates(subset=filter_subset).reset_index(drop=True)
df.columns

Index(['RawMessage', 'MessageCategory', '|Count', ':Count', 'last|', '[anim]',
       '[block]', '[broken]', '[consumed]', '[damage]', '[eat]', '[fail]',
       '[fatigue]', '[from]', '[fromitem]', '[identify]', '[miss]', '[move]',
       '[msg]', '[notarget]', '[of]', '[partiallytrapped]', '[silent]',
       '[source]', '[spread]', '[still]', '[true]', '[upkeep]', '[weak]',
       '[weaken]', '[wisher]', 'Generation'],
      dtype='object')

In [5]:
parse_success = []
err_state = []
for raw in df["RawMessage"]:
    bm = BattleMessage.from_message(raw)

    err_state.append(bm.ERR_STATE)
    parse_success.append(BMType("unknown") != bm.BMTYPE)

df["ParseSuccess"] = parse_success
df["ErrorState"] = err_state
df

,RawMessage,MessageCategory,|Count,:Count,last|,[anim],[block],[broken],[consumed],[damage],...,[spread],[still],[true],[upkeep],[weak],[weaken],[wisher],Generation,ParseSuccess,ErrorState
195,|-ability|p2a: Granbull|Intimidate|boost,-ability,4,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,3,True,None
203,|-ability|p1a: Deoxys|Pressure|[silent],-ability,4,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,3,True,None
260,|-ability|p2a: Gardevoir|Clear Body|[from] abi...,-ability,5,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,3,True,None
1520,|-ability|p1a: Suicune|Pressure|[silent],-ability,4,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,3,True,None
1522,|-ability|p2b: Porygon2|Inner Focus|[from] abi...,-ability,5,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,3,True,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,|win|colress-gpt,win,2,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,7,True,None
2160,|win|colress-gpt-test1,win,2,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,7,True,None
1013,|win|colress-gpt,win,2,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,8,True,None
2387,|win|colress-gpt,win,2,0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,8,True,None


In [6]:
f'Progress: {100 * (df["ParseSuccess"].sum() / df.shape[0]):.2f}%'

'Progress: 100.00%'

In [7]:
df[~df["ParseSuccess"]]["MessageCategory"].value_counts()

Series([], Name: count, dtype: int64)

In [8]:
df[~df["ParseSuccess"]]["ErrorState"].value_counts()

Series([], Name: count, dtype: int64)

In [9]:
m_cat = "request"

df[(df["MessageCategory"] == m_cat) & ~(df["ParseSuccess"])]

,RawMessage,MessageCategory,|Count,:Count,last|,[anim],[block],[broken],[consumed],[damage],...,[spread],[still],[true],[upkeep],[weak],[weaken],[wisher],Generation,ParseSuccess,ErrorState


In [10]:
df[(df["MessageCategory"] == m_cat) & ~(df["ParseSuccess"])]["RawMessage"].to_list()

[]

In [17]:
df[(df["MessageCategory"] == m_cat) & ((df["Generation"] == 8))]["RawMessage"].to_list()

['|request|',
 '|request|{"teamPreview":true,"side":{"name":"colress-gpt","id":"p1","pokemon":[{"ident":"p1: Wailord","details":"Wailord, L90, F","condition":"452/452","active":true,"stats":{"atk":167,"def":132,"spa":213,"spd":132,"spe":159},"moves":["waterspout","hypervoice","icebeam","hydropump"],"baseAbility":"oblivious","item":"choicespecs","pokeball":"pokeball","ability":"oblivious"},{"ident":"p1: Cramorant","details":"Cramorant, L84, M","condition":"255/255","active":false,"stats":{"atk":191,"def":141,"spa":191,"spd":208,"spe":191},"moves":["roost","superpower","bravebird","surf"],"baseAbility":"gulpmissile","item":"heavydutyboots","pokeball":"pokeball","ability":"gulpmissile"},{"ident":"p1: Falinks","details":"Falinks, L84","condition":"246/246","active":false,"stats":{"atk":216,"def":216,"spa":166,"spd":149,"spe":174},"moves":["poisonjab","rockslide","noretreat","closecombat"],"baseAbility":"defiant","item":"lifeorb","pokeball":"pokeball","ability":"defiant"},{"ident":"p1: Tapu